In [28]:
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Preparação dos dados 

In [29]:
diretorio_base = "/home/breno-luiz/Documents/Planilha/dados_limpos/"
caminho_dados = os.path.join(diretorio_base, "dados_completos.csv")

try:
    df = pd.read_csv(caminho_dados)
    print("✅ Dados carregados com sucesso!")
    print("📋 Colunas disponíveis:", df.columns.tolist())
except Exception as e:
    print(f"❌ Erro ao carregar dados: {e}")
    raise

df['data_venda'] = pd.to_datetime(df['data_venda'], errors='coerce', dayfirst=True)
df = df.dropna(subset=['data_venda'])
df['mes_ano'] = df['data_venda'].dt.strftime('%Y-%m')
df['dia_semana'] = df['data_venda'].dt.day_name()
dias_ordem = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

df['valor_total_sem_desconto'] = df['valor_monetario_total'] + df['valor_desconto']
df['desconto_percentual'] = (df['valor_desconto'] / df['valor_total_sem_desconto']) * 100
df['lucro_unitario'] = df['valor_monetario_total'] - (df['quantidade'] * df['custo_produto_unitario'])

def resumir_nome(nome, limite=15):
    return nome[:limite] + "..." if len(nome) > limite else nome

df["nomeproduto_resumido"] = df["descricaoproduto"].apply(lambda x: resumir_nome(str(x), 15))
df["familia_resumida"] = df["descricaofamilia"].apply(lambda x: resumir_nome(str(x), 15))

meses_disponiveis = sorted(df['mes_ano'].unique())
familias_disponiveis = sorted(df['familia_resumida'].unique())

/tmp/ipykernel_133326/1080060821.py:5: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



✅ Dados carregados com sucesso!
📋 Colunas disponíveis: ['codigo_contrato', 'data_venda', 'codigo_produto', 'filial_venda', 'quantidade', 'valor_monetario_total', 'valor_desconto', 'codigo_vendedor', 'codigo_cliente', 'descricaoproduto', 'codigo_familia', 'preco_venda_unitario', 'custo_produto_unitario', 'descricaofamilia', 'nome_vendedor', 'mes_ano', 'valor_bruto']


## Configuração

In [30]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])

navbar = dbc.NavbarSimple(
    brand="Dashboard Comercial - Lumen Store",
    brand_href="#",
    color="dark",
    dark=True,
    sticky="top",
)

def criar_card_kpi(titulo, valor, icone):
    return dbc.Card(
        dbc.CardBody([
            html.H6(titulo, className="card-title text-uppercase", style={"fontWeight": "bold", "marginBottom": "0px"}),
            html.H3(valor, className="card-text", style={"marginBottom": "0px"}),
            html.Span(icone, className="fs-2", style={"marginTop": "5px"})
        ]),
        className="text-center shadow-sm m-2",
        style={"backgroundColor": "#2a2e33", "color": "#ffffff", "border": "none"}
    )
app.layout = dbc.Container([
    navbar,
    
    dbc.Row(
        dbc.Col([
            html.H2("Painel Executivo", className="text-center my-3"),
            html.P("Visualize os principais indicadores do desempenho comercial.", 
                className="text-center lead")
        ], width=12)
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("🔍 Filtro Temporal", className="h5"),
                dbc.CardBody(
                    dcc.Dropdown(
                        id="mes-dropdown",
                        options=[{'label': m, 'value': m} for m in meses_disponiveis],
                        value=meses_disponiveis[-1] if meses_disponiveis else None,
                        clearable=False,
                        style={"color": "#000000"}
                    )
                ),
                dbc.CardFooter("Selecione o período para análise")
            ], className="shadow-lg mb-4"),
            width=12
        )
    ),
    
    dbc.Row(id="kpi-cards", className="mb-4", justify="center"),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("📈 Evolução Histórica", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-evolucao", config={"displayModeBar": False})),
                dbc.CardFooter("Identificar meses com maior faturamento para direcionar estratégias de vendas")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("🏪 Participação por Filial", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-filial", config={"displayModeBar": False})),
                dbc.CardFooter("Analisar a eficácia de cada filial e identificar aquelas com menor participação para intervenção")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("💰 Margem por Categoria", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-margem", config={"displayModeBar": False})),
                dbc.CardFooter("Analisar a lucratividade média por família para verificar se o desempenho do grupo está alinhado com seu volume de vendas")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("🏷️ Top 5 Descontos", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-descontos", config={"displayModeBar": False})),
                dbc.CardFooter("Avaliar os produtos que recebem os maiores descontos para identificar potenciais perdas de margem e ajustar a política promocional")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("🚀 Produtos Mais Vendidos", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-vendas", config={"displayModeBar": False})),
                dbc.CardFooter("Identificar os produtos com maior volume de vendas para ajustar estratégias de estoque")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("🗓️ Padrão Semanal de Vendas", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-heatmap", config={"displayModeBar": False})),
                dbc.CardFooter("Analisar os dias com maior e menor movimento para otimizar campanhas e a logística de atendimento")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("📊 Participação por Família (Curva Pareto)", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-pareto", config={"displayModeBar": False})),
                dbc.CardFooter("Entender a contribuição de cada família para o faturamento e focar nas mais relevantes")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("🔝 Top 5 Produtos por Família", className="h5"),
                dbc.CardBody([
                    dcc.Dropdown(
                        id="familia-dropdown",
                        options=[{'label': fam, 'value': fam} for fam in familias_disponiveis],
                        value=familias_disponiveis[0] if familias_disponiveis else None,
                        clearable=False,
                        style={"color": "#000000"},
                        className="mb-3"
                    ),
                    dcc.Graph(id="grafico-top-produtos", config={"displayModeBar": False})
                ]),
                dbc.CardFooter("Destacar produtos que se sobressaem em cada categoria para ajustar estoque e campanhas")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("📈 Crescimento/Declínio das Famílias", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-crescimento", config={"displayModeBar": False})),
                dbc.CardFooter("Detectar tendências de crescimento ou queda para priorizar investimentos e ações de marketing")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("💹 Margem de Lucro: Produtos vs Famílias", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-lucro", config={"displayModeBar": False})),
                dbc.CardFooter("Identificar quais produtos e famílias geram maior lucro, mesmo que não tenham o maior volume de vendas")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("🏷️ Desconto Médio por Família", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-desconto-familia", config={"displayModeBar": False})),
                dbc.CardFooter("Compreender como os descontos variam entre famílias para ajustar a estratégia de preços e melhorar a margem de lucro")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("💳 Ticket Médio por Canal de Venda (SAH)", className="h5"),
                dbc.CardBody(dcc.Graph(id="grafico-ticket-medio", config={"displayModeBar": False})),
                dbc.CardFooter("Monitorar a média de receita por pedido para identificar oportunidades de aumento do ticket médio")
            ], className="shadow-sm mb-4"),
            width=12
        )
    ),
    
    dbc.Row(
        dbc.Col(
            html.Footer([
                html.P("© 2025 Lumen Store | Desenvolvido com Dash, Plotly e Bootstrap", 
                    className="text-center text-muted", style={"fontSize": "0.9rem"})
            ], style={"padding": "10px", "marginTop": "20px", "borderTop": "1px solid #444"})
        )
    )
    
], fluid=True, style={"padding": "20px", "backgroundColor": "#1c1e21"})

/home/breno-luiz/Documents/EQUALS/.venv/lib/python3.12/site-packages/dash/dash.py:550: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



## Templates

In [31]:
@app.callback(
    Output("kpi-cards", "children"),
    Input("mes-dropdown", "value")
)
def update_kpi_cards(mes_selecionado):
    df_filtrado = df[df['mes_ano'] == mes_selecionado]
    total_faturamento = df_filtrado['valor_monetario_total'].sum()
    media_desconto = df_filtrado['desconto_percentual'].mean()
    total_vendas = df_filtrado['quantidade'].sum()
    
    kpi1 = criar_card_kpi("Faturamento Total (R$)", f"{total_faturamento:,.2f}", "💰")
    kpi2 = criar_card_kpi("Média de Desconto (%)", f"{media_desconto:,.2f}", "🏷️")
    kpi3 = criar_card_kpi("Total de Vendas", f"{total_vendas:,}", "🚀")
    
    return dbc.Row([dbc.Col(kpi1, md=4), dbc.Col(kpi2, md=4), dbc.Col(kpi3, md=4)], justify="center")

@app.callback(
    [Output("grafico-evolucao", "figure"),
    Output("grafico-filial", "figure"),
    Output("grafico-margem", "figure"),
    Output("grafico-descontos", "figure"),
    Output("grafico-vendas", "figure"),
    Output("grafico-heatmap", "figure"),
    Output("grafico-pareto", "figure"),
    Output("grafico-ticket-medio", "figure")],
    Input("mes-dropdown", "value")
)
def update_graphs(mes_selecionado):
    df_filtrado = df[df['mes_ano'] == mes_selecionado]
    
    evolucao = df.groupby('mes_ano')['valor_monetario_total'].sum().reset_index()
    fig1 = px.line(evolucao, x='mes_ano', y='valor_monetario_total',
                title="Evolução Mensal do Faturamento",
                labels={'mes_ano': 'Mês', 'valor_monetario_total': 'Faturamento (R$)'})
    fig1.update_traces(hovertemplate="<b>Mês:</b> %{x}<br><b>Faturamento:</b> R$ %{y:,.2f}<extra></extra>")
    fig1.update_layout(hovermode="x unified", template="plotly_dark")
    
    participacao = df_filtrado.groupby('filial_venda')['valor_monetario_total'].sum().reset_index()
    fig2 = px.pie(participacao, values='valor_monetario_total', names='filial_venda',
                title=f"Distribuição por Filial - {mes_selecionado}", hole=0.3, template="plotly_dark")
    fig2.update_traces(hovertemplate="<b>Filial:</b> %{label}<br><b>Faturamento:</b> R$ %{value:,.2f}<extra></extra>")
    
    margem = df_filtrado.groupby('familia_resumida')['lucro_unitario'].mean().reset_index()
    fig3 = px.bar(margem.sort_values('lucro_unitario'),
                x='lucro_unitario', y='familia_resumida',
                title=f"Margem Média por Categoria - {mes_selecionado}",
                labels={'lucro_unitario': 'Lucro Unitário (R$)', 'familia_resumida': 'Família'},
                orientation='h',
                color='lucro_unitario',
                color_continuous_scale='Blues',
                template="plotly_dark")
    fig3.update_traces(hovertemplate="<b>Família:</b> %{y}<br><b>Lucro Unitário:</b> R$ %{x:,.2f}<extra></extra>")
    
    descontos = (df_filtrado.groupby('nomeproduto_resumido')['desconto_percentual']
                .mean().nlargest(5).reset_index())
    fig4 = px.bar(descontos, x='desconto_percentual', y='nomeproduto_resumido',
                title=f"Top 5 Maiores Descontos - {mes_selecionado}",
                labels={'desconto_percentual': 'Desconto Médio (%)', 'nomeproduto_resumido': 'Produto'},
                orientation='h',
                color='desconto_percentual',
                color_continuous_scale='Reds',
                template="plotly_dark")
    fig4.update_traces(hovertemplate="<b>Produto:</b> %{y}<br><b>Desconto Médio:</b> %{x:.2f}%<extra></extra>")
    
    vendas = (df_filtrado.groupby('nomeproduto_resumido')['quantidade']
            .sum().nlargest(5).reset_index())
    fig5 = px.bar(vendas, x='quantidade', y='nomeproduto_resumido',
                title=f"Top 5 Produtos Mais Vendidos - {mes_selecionado}",
                labels={'quantidade': 'Unidades Vendidas', 'nomeproduto_resumido': 'Produto'},
                orientation='h',
                color='quantidade',
                color_continuous_scale='Greens',
                template="plotly_dark")
    fig5.update_traces(hovertemplate="<b>Produto:</b> %{y}<br><b>Unidades Vendidas:</b> %{x}<extra></extra>")
    
    heatmap_data = (df_filtrado.pivot_table(index='dia_semana',
                                            columns='filial_venda',
                                            values='valor_monetario_total',
                                            aggfunc='sum')
                    .reindex(dias_ordem))
    fig6 = px.imshow(heatmap_data,
                    title=f"Vendas por Dia da Semana - {mes_selecionado}",
                    labels=dict(x="Filial", y="Dia da Semana", color="Vendas (R$)"),
                    color_continuous_scale='Purples',
                    aspect="auto",
                    template="plotly_dark")
    fig6.update_xaxes(side="top")
    
    faturamento_familia = (df_filtrado.groupby('familia_resumida')['valor_monetario_total']
                        .sum().reset_index().sort_values(by='valor_monetario_total', ascending=False))
    faturamento_familia['cumulative'] = faturamento_familia['valor_monetario_total'].cumsum()
    faturamento_familia['percentual_cum'] = 100 * faturamento_familia['cumulative'] / faturamento_familia['valor_monetario_total'].sum()
    fig7 = go.Figure()
    fig7.add_bar(x=faturamento_familia['familia_resumida'], y=faturamento_familia['valor_monetario_total'], name='Faturamento',
                hovertemplate="<b>Família:</b> %{x}<br><b>Faturamento:</b> R$ %{y:,.2f}<extra></extra>")
    fig7.add_scatter(x=faturamento_familia['familia_resumida'], y=faturamento_familia['percentual_cum'],
                    mode='lines+markers', name='Percentual Acumulado', yaxis='y2',
                    hovertemplate="<b>Família:</b> %{x}<br><b>Percentual Acumulado:</b> %{y:.2f}%<extra></extra>")
    fig7.update_layout(
        title=f"Participação de Famílias no Faturamento - {mes_selecionado}",
        yaxis=dict(title="Faturamento (R$)"),
        yaxis2=dict(title="Percentual Acumulado (%)", overlaying='y', side='right'),
        template="plotly_dark"
    )
    ticket_df = df_filtrado.groupby('filial_venda').agg({
        'valor_monetario_total': 'sum',
        'codigo_contrato': 'nunique'
    }).reset_index()
    ticket_df['ticket_medio'] = ticket_df['valor_monetario_total'] / ticket_df['codigo_contrato']
    fig_ticket = px.bar(ticket_df, x='ticket_medio', y='filial_venda', orientation='h',
                        title=f"Ticket Médio por Canal de Venda - {mes_selecionado}",
                        labels={'filial_venda': 'Canal de Venda', 'ticket_medio': 'Ticket Médio (R$)'},
                        color='ticket_medio', color_continuous_scale='Blues',
                        template="plotly_dark")
    fig_ticket.update_traces(hovertemplate="<b>Canal:</b> %{y}<br><b>Ticket Médio:</b> R$ %{x:,.2f}<extra></extra>")
    
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig_ticket

@app.callback(
    Output("grafico-top-produtos", "figure"),
    Input("familia-dropdown", "value")
)
def update_top_produtos(selected_familia):
    df_fam = df[df['familia_resumida'] == selected_familia]
    top_prod = (df_fam.groupby('nomeproduto_resumido')['quantidade']
                .sum().nlargest(5).reset_index())
    fig = px.bar(top_prod, x='quantidade', y='nomeproduto_resumido', orientation='h',
                title=f"Top 5 Produtos na Família {selected_familia}",
                labels={'quantidade': 'Unidades Vendidas', 'nomeproduto_resumido': 'Produto'},
                color='quantidade', color_continuous_scale='Greens',
                template="plotly_dark")
    fig.update_traces(hovertemplate="<b>Produto:</b> %{y}<br><b>Unidades Vendidas:</b> %{x}<extra></extra>")
    return fig

@app.callback(
    Output("grafico-crescimento", "figure"),
    Input("mes-dropdown", "value")
)
def update_crescimento(_):
    familia_mensal = df.groupby(['mes_ano', 'familia_resumida'])['valor_monetario_total'].sum().reset_index()
    growth_data = []
    for fam in familia_mensal['familia_resumida'].unique():
        df_fam = familia_mensal[familia_mensal['familia_resumida'] == fam].sort_values('mes_ano')
        if df_fam.iloc[0]['valor_monetario_total'] > 0:
            growth = ((df_fam.iloc[-1]['valor_monetario_total'] - df_fam.iloc[0]['valor_monetario_total']) / df_fam.iloc[0]['valor_monetario_total']) * 100
            growth_data.append({'familia_resumida': fam, 'growth': growth})
    growth_df = pd.DataFrame(growth_data).sort_values('growth', ascending=False)
    fig = px.bar(growth_df, x='growth', y='familia_resumida', orientation='h',
                title="Crescimento/Declínio das Famílias ao Longo do Tempo",
                labels={'growth': 'Crescimento (%)', 'familia_resumida': 'Família'},
                color='growth', color_continuous_scale='RdYlGn',
                template="plotly_dark")
    fig.update_traces(hovertemplate="<b>Família:</b> %{y}<br><b>Crescimento:</b> %{x:.2f}%<extra></extra>")
    return fig

@app.callback(
    Output("grafico-lucro", "figure"),
    Input("mes-dropdown", "value")
)
def update_lucro(_):
    produto_lucro = (df.groupby('nomeproduto_resumido')['lucro_unitario']
                    .sum().reset_index().nlargest(5, 'lucro_unitario'))
    familia_lucro = (df.groupby('familia_resumida')['lucro_unitario']
                    .sum().reset_index().nlargest(5, 'lucro_unitario'))
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Top Produtos por Lucro", "Top Famílias por Lucro"))
    fig.add_trace(go.Bar(x=produto_lucro['lucro_unitario'], y=produto_lucro['nomeproduto_resumido'],
                        orientation='h', marker=dict(color='indianred'),
                        hovertemplate="<b>Produto:</b> %{y}<br><b>Lucro:</b> R$ %{x:,.2f}<extra></extra>"),
                row=1, col=1)
    fig.add_trace(go.Bar(x=familia_lucro['lucro_unitario'], y=familia_lucro['familia_resumida'],
                        orientation='h', marker=dict(color='royalblue'),
                        hovertemplate="<b>Família:</b> %{y}<br><b>Lucro:</b> R$ %{x:,.2f}<extra></extra>"),
                row=1, col=2)
    fig.update_layout(title_text="Margem de Lucro: Contribuição de Produtos e Famílias", template="plotly_dark")
    return fig

@app.callback(
    Output("grafico-desconto-familia", "figure"),
    Input("mes-dropdown", "value")
)
def update_desconto_familia(mes_selecionado):
    df_filtrado = df[df['mes_ano'] == mes_selecionado]
    desconto_familia = df_filtrado.groupby('familia_resumida')['desconto_percentual'].mean().reset_index()
    fig = px.bar(desconto_familia, x='desconto_percentual', y='familia_resumida', orientation='h', 
                title=f"Desconto Médio por Família - {mes_selecionado}", 
                labels={'desconto_percentual': 'Desconto Médio (%)', 'familia_resumida': 'Família'},
                color='desconto_percentual', color_continuous_scale='Reds',
                template="plotly_dark")
    fig.update_traces(hovertemplate="<b>Família:</b> %{y}<br><b>Desconto:</b> %{x:.2f}%<extra></extra>")
    return fig

## Execução

In [32]:
if __name__ == '__main__':
    app.run(mode='inline', port=8050)